In [1]:
import pandas as pd
import networkx as nx
from collections import Counter

# 读取xlsx文件
data = pd.read_excel(r'NewTAC1.xlsx')

In [2]:
data['IPC主分类号'] = data['IPC主分类号'].str[:4]
data['[标]原始申请(专利权)人'] = data['[标]原始申请(专利权)人'].str.split('|').str[0]
data['被引专利权人'] = data['被引专利权人'].str.upper()
data['[标]原始申请(专利权)人'] = data['[标]原始申请(专利权)人'].str.upper()
data = data.drop([0, 1])
data = data.fillna('-')
data['[标]原始申请(专利权)人'] = data['[标]原始申请(专利权)人'].str.title()
data['被引专利权人'] = data['被引专利权人'].str.title()
data

,公开(公告)号,标题,法律状态/事件,申请日,发明人,申请号,权利要求,简单同族,简单同族成员数量,摘要,...,被引用专利,被引用专利数量,引用专利,引用专利数量,非专利引文数量,许可类型,5年内被引用次数,诉讼案件数,质押人,被引专利权人
2,US675631A,Thio-urea compound and process of making same.,期限届满,1900-12-31,ADOLF ISRAEL | RICHARD KOTHE,US1900041628,Having now described our invention and in 20 w...,US675631A,1,-,...,US5144064A,-,-,0,0,-,0,0,-,Basf Se
3,US20230250275A1,Algae thermoplastic composition and process of...,公开,2023-03-07,"ZELLER, MARK ASHTON | HUNT, RYAN",US18/180014,1. An algae-based thermoplastic composition co...,US20230250275A1,1,An algae-based thermoplastic foam is provided ...,...,-,-,-,0,0,-,0,0,-,-
4,US20230212537A1,Novel esterases and uses thereof,公开,2022-12-22,"DAVID, BENOÎT | ANDRE, ISABELLE | BEN KHALED, ...",US18/145073,1. An esterase which (i) has at least 95% iden...,CA3107517A1 | CN112654701A | EP3830254A1 | IN2...,10,"The present invention relates to esterases, mo...",...,-,-,-,0,0,-,0,0,-,-
5,US20230127747A1,Biodegradable and Industrially Compostable Inj...,实质审查 | 权利转移,2022-12-23,"FALKEN, ROBERT",US18/087934,1-20. (canceled)_x000D_\n21. A method for manu...,AU2019272664A1 | BR112020023590A2 | CN11251277...,12,A process for injection molded microcellular f...,...,-,-,US20100198133A1 | US20130154143A1,2,0,-,0,0,-,-
6,US20230208143A1,System and method for smoothing photovoltaic g...,公开,2022-12-22,"AREDES, MAURICIO | BARBOSA ROLIM, LUÍS GUILHER...",US18/087199,1. A system for smoothing photovoltaic generat...,BR102021026269A2 | US20230208143A1,2,The present invention provides a system and me...,...,-,-,-,0,0,-,0,0,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,EP2294125A2,Starch-containing thermoplastic or elastomer c...,撤回-视为撤回,2009-06-12,"MENTINK, LÉON | GIMENEZ, JÉRÔME | LAGNEAUX, DI...",EP2009761936,REVENDICATIONS_x000D_\n1. Composition thermopl...,AU2009259118A1 | BRPI0914417A2 | CA2726860A1 |...,9,The invention relates to a thermoplastic and/o...,...,-,-,-,0,0,-,0,0,-,-
446,EP2247661A2,Method for preparing thermoplastic composition...,撤回-视为撤回,2009-01-29,"MENTINK, LÉON | LAGNEAUX, DIDIER | GIMENEZ, JÉ...",EP2009705988,REVENDICATIONS_x000D_\n1. Composition à base d...,AU2009208830A1 | AU2009208830B2 | BRPI0907038A...,13,The subject matter of the present invention is...,...,-,-,-,0,0,-,0,0,-,-
447,EP2144221B1,Biodegradable film for advertisement,权利终止,2009-07-07,"LEE, IN KEE | KIM, SEONG JIN",EP2009008878,"1. A biodegradable film for advertisement, co...",AT863302T | EP2144221A1 | EP2144221B1 | JP2010...,6,The present invention relates to a biodegradab...,...,CN103282589A | CN103282589B | CN105173383A | C...,-,EP1116748A1 | JP2004188884A | JP2007262189A | ...,4,1,-,5,0,-,Lg Group | Lg Group | Jiangsu Red Sun New Mate...
448,EP2144966A1,Coextrusion binders on a renewable/biodegradab...,授权,2008-05-14,"CHOPINEZ, FABRICE | LAFFARGUE, JOHANN | PRADEL...",EP2008805782,19_x000D_\nREVENDICATIONS_x000D_\n1. Compositi...,AT592757T | CN101679731A | CN101679731B | DE60...,13,The invention relates to coextrusion binders i...,...,-,-,-,0,0,-,0,0,-,-


In [1]:
# 构建边列表
edges = []
for row in data.iterrows():
    _, patent = row
    cited_patents = str(patent['被引专利权人']).split(' | ')
    for cited_patent in cited_patents:
        if cited_patent != '-':
            edges.append((patent['公开(公告)号'], cited_patent))

# 统计第二列中每个字符串的数量
string_counts = Counter(edge[0] for edge in edges)

# 仅保留字符串数量大于等于2的行
filtered_edges = [edge for edge in edges if string_counts[edge[0]] >= 10]

# 创建有向图
G = nx.DiGraph()
G.add_edges_from(filtered_edges)
G_patent = G.reverse()
# 生成带权邻接矩阵
adj_matrix = nx.to_pandas_adjacency(G_patent, weight='weight')

# 保存邻接矩阵到CSV文件
adj_matrix.to_excel("专利被引用网络带权邻接矩阵10专利权人.xlsx")

NameError: name 'data' is not defined

In [4]:
clustering_coefficients = nx.clustering(G)  # 计算每个节点的聚类系数
# 如果需要，可以打印每个节点的聚类系数
for node, cc in clustering_coefficients.items():
    print(f"Node {node}: Clustering Coefficient = {cc}")

Node O2 Partners Llc: Clustering Coefficient = 0.019927536231884056
Node Carbios: Clustering Coefficient = 0.006493506493506494
Node Hanbio Co Ltd: Clustering Coefficient = 0
Node Biologiq, Inc: Clustering Coefficient = 0.13333333333333333
Node Basf Se: Clustering Coefficient = 0.01282051282051282
Node Guangdong Huifa Plastic Industry Technology Co Ltd: Clustering Coefficient = 0
Node King Abdulaziz University: Clustering Coefficient = 0
Node Individual: Clustering Coefficient = 0.017786561264822136
Node South China University Of Technology Scut: Clustering Coefficient = 0
Node Carbiolice: Clustering Coefficient = 0
Node Tbm Co Ltd: Clustering Coefficient = 0
Node Jiangsu Sanlian New Materials Co., Ltd.: Clustering Coefficient = 0
Node Anqing Qianqian Paper Co Ltd: Clustering Coefficient = 0
Node Luoyang Lvzhihui Plastic Degradation Technology Co Ltd: Clustering Coefficient = 0
Node Henan Qingyuan Tianren Biotechnology Co Ltd: Clustering Coefficient = 0
Node Fast Thinking Technology Sh

In [5]:
average_clustering = nx.average_clustering(G)  # 计算整个图的平均聚类系数
print(f"Average Clustering Coefficient = {average_clustering}")

Average Clustering Coefficient = 0.005726292442300018


In [4]:
betweenness_centrality = nx.betweenness_centrality(G_patent)
closeness_centrality = nx.closeness_centrality(G_patent)

# 将指标结果转换为DataFrame
bc_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Node', 'Betweenness Centrality'])
cc_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Node', 'Closeness Centrality'])

# 保存指标结果为CSV文件
bc_df.to_excel("betweenness_centrality_patent.xlsx", index=False)
cc_df.to_excel("closeness_centrality_patent.xlsx", index=False)

In [5]:
import pandas as pd

# 计算每个节点的入度和出度
in_degrees = G_patent.in_degree()
out_degrees = G_patent.out_degree()
# 创建 DataFrame 存储节点、入度和出度信息
node_list = list(G_patent.nodes())
degree_data = {'Node': node_list, 'In-Degree': [in_degrees[node] for node in node_list], 'Out-Degree': [out_degrees[node] for node in node_list]}
degree_df = pd.DataFrame(degree_data)

# 保存 DataFrame 到 Excel 文件
degree_df.to_excel("节点度信息.xlsx", index=False)